# Table of Contents
1. [Basic autograd example 1](#Basic-autograd-example-1)
2. [Basic autograd example 2](#Basic-autograd-example-2)
3. [Loading data from numpy](#Loading-data-from-numpy)               
4. [Implementing the input pipline](#Implementing-the-input-pipline)       
5. [Input pipline for custom dataset](#Input-pipline-for-custom-dataset)       
6. [Using pretrained model](#Using-pretrained-model)              
7. [Save and load model](#Save-and-load-model)               

In [1]:
import torch 
import torchvision
import torch.nn as nn
import numpy as np
import torch.utils.data as data
import torchvision.transforms as transforms
import torchvision.datasets as dsets
from torch.autograd import Variable

##  Basic autograd example 1 

Create tensors.

In [34]:
x = Variable(torch.Tensor([1]), requires_grad=True)
w = Variable(torch.Tensor([2]), requires_grad=True)
b = Variable(torch.Tensor([3]), requires_grad=True)

Build a computational graph.

In [35]:
y = w * x + b    # y = 2 * x + 3

Compute gradients.

In [36]:
y.backward()

Print out the gradients.

In [37]:
print(x.grad)    # x.grad = 2 
print(w.grad)    # w.grad = 1 
print(b.grad)    # b.grad = 1 

Variable containing:
 2
[torch.FloatTensor of size 1]

Variable containing:
 1
[torch.FloatTensor of size 1]

Variable containing:
 1
[torch.FloatTensor of size 1]



In [38]:
print(x,w,b)

Variable containing:
 1
[torch.FloatTensor of size 1]
 Variable containing:
 2
[torch.FloatTensor of size 1]
 Variable containing:
 3
[torch.FloatTensor of size 1]



## Basic autograd example 2 

In [39]:
# Create tensors.
x = Variable(torch.randn(5, 3))
y = Variable(torch.randn(5, 2))
print(x)
print(y)

Variable containing:
-2.2296  0.4541 -0.4135
-0.5685  0.6146  0.3912
-0.3639 -1.4462 -0.6820
 0.1249  0.7281 -0.4214
 0.4675 -0.3043  2.0433
[torch.FloatTensor of size 5x3]

Variable containing:
-0.2260 -0.4310
 0.2594  1.5715
-0.1659 -2.0972
-0.3871  0.9333
 0.9648 -0.6202
[torch.FloatTensor of size 5x2]



In [40]:
# Build a linear layer.
linear = nn.Linear(3, 2)
print ('w: ', linear.weight)
print ('b: ', linear.bias)

w:  Parameter containing:
 0.4986  0.5133  0.3568
 0.4597 -0.4259 -0.2075
[torch.FloatTensor of size 2x3]

b:  Parameter containing:
-0.0696
-0.4606
[torch.FloatTensor of size 2]



In [41]:
# Build Loss and Optimizer.
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(linear.parameters(), lr=0.01)

In [42]:
# Forward propagation.
pred = linear(x)
print(pred)

Variable containing:
-1.0957 -1.5931
 0.1020 -1.0649
-1.2368  0.1295
 0.2160 -0.6258
 0.7362 -0.5401
[torch.FloatTensor of size 5x2]



In [43]:
# Compute loss.
loss = criterion(pred, y)
print('loss: ', loss.data[0])

loss:  1.8040215969085693


In [44]:
# Backpropagation.
loss.backward()

In [45]:
# Print out the gradients.
print ('dL/dw: ', linear.weight.grad) 
print ('dL/db: ', linear.bias.grad)

dL/dw:  Variable containing:
 0.4774  0.3131  0.0614
 0.6244 -1.3056 -0.2498
[torch.FloatTensor of size 2x3]

dL/db:  Variable containing:
-0.3447
-0.6102
[torch.FloatTensor of size 2]



In [46]:
# 1-step Optimization (gradient descent).
optimizer.step()

In [47]:
# You can also do optimization at the low level as shown below.
# linear.weight.data.sub_(0.01 * linear.weight.grad.data)
# linear.bias.data.sub_(0.01 * linear.bias.grad.data)

# Print out the loss after optimization.
pred = linear(x)
loss = criterion(pred, y)
print('loss after 1 step optimization: ', loss.data[0])

loss after 1 step optimization:  1.7744073867797852


## Loading data from numpy 

In [22]:
a = np.array([[1,2], [3,4]])
b = torch.from_numpy(a)      # convert numpy array to torch tensor
c = b.numpy()                # convert torch tensor to numpy array

## Implementing the input pipline

In [48]:
# Download and construct dataset.
train_dataset = dsets.CIFAR10(root='/data/wanggh/data/cifar10/',
                               train=True, 
                               transform=transforms.ToTensor(),
                               download=True)

Files already downloaded and verified


In [49]:
# Select one data pair (read data from disk).
image, label = train_dataset[0]
print (image.size())
print (label)

torch.Size([3, 32, 32])
6


In [50]:
# Data Loader (this provides queue and thread in a very simple way).
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=100, 
                                           shuffle=True,
                                           num_workers=2)

In [26]:
# When iteration starts, queue and thread start to load dataset from files.
data_iter = iter(train_loader)

# Mini-batch images and labels.
images, labels = data_iter.next()

# Actual usage of data loader is as below.
for images, labels in train_loader:
    # Your training code will be written here
    pass

## Input pipline for custom dataset

In [28]:
# You should build custom dataset as below.
class CustomDataset(data.Dataset):
    def __init__(self):
        # TODO
        # 1. Initialize file path or list of file names. 
        pass
    def __getitem__(self, index):
        # TODO
        # 1. Read one data from file (e.g. using numpy.fromfile, PIL.Image.open).
        # 2. Preprocess the data (e.g. torchvision.Transform).
        # 3. Return a data pair (e.g. image and label).
        pass
    def __len__(self):
        # You should change 0 to the total size of your dataset.
        return 0 

In [29]:
# Then, you can just use prebuilt torch's data loader. 
custom_dataset = CustomDataset()
train_loader = torch.utils.data.DataLoader(dataset=custom_dataset,
                                           batch_size=100, 
                                           shuffle=True,
                                           num_workers=2)

## Using pretrained model

In [30]:
# Download and load pretrained resnet.
resnet = torchvision.models.resnet18(pretrained=True)

# If you want to finetune only top layer of the model.
for param in resnet.parameters():
    param.requires_grad = False
    
# Replace top layer for finetuning.
resnet.fc = nn.Linear(resnet.fc.in_features, 100)  # 100 is for example.

# For test.
images = Variable(torch.randn(10, 3, 224, 224))
outputs = resnet(images)
print (outputs.size())   # (10, 100)

torch.Size([10, 100])


## Save and load the model

In [31]:
# Save and load the entire model.
torch.save(resnet, 'model.pkl')
model = torch.load('model.pkl')

# Save and load only the model parameters(recommended).
torch.save(resnet.state_dict(), 'params.pkl')
resnet.load_state_dict(torch.load('params.pkl'))

In [33]:
print(resnet)

ResNet(
  (conv1): Conv2d (3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), dilation=(1, 1))
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d (64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d (64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d (64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d (64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNo